# In this notebook I scraped my data from the two subreddits I have chosen, Alien Abduction and Sleep Paralysis.  I cleaned each dataset and read those to a csv, then I concated the datasets for modeling in notebook 2. Also, I copied the results from my web scraping into a markdown cell so you dont have to run it.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import seaborn as sns
import missingno as msno
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from bs4 import BeautifulSoup
import time
import requests
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
import praw
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re
from IPython.core.interactiveshell import InteractiveShell
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [ ]:
InteractiveShell.ast_node_interactivity = "all"

In [2]:
url='https://www.reddit.com/'
res=requests.get(url)

In [3]:
reddit = praw.Reddit(client_id='qPz4QzW8_2s4eA', client_secret='hfYoCLBMIyGoFFt07mJGndLMBKg', user_agent='Reddit_JTJ2')

### get 10 hot posts from Sleepparalysis key word to make sure i have the correct subreddit

In [ ]:

#hot_posts_sp = reddit.subreddit('Sleepparalysis').hot(limit=10)
#for post in hot_posts_sp:
#    print(post.title)



Identifying SP
NSFW: My sleep paralysis experiences with rape and disturbing hallucinations. I’m experiencing this almost every night.
The Times I Have Experienced Sleep Paralysis So Far
I think I may have just had a sleep paralysis and it was terrifying.
Defying the laws of physics - sleep paralysis?
I had very weird sleep paralysis
Few times a week lately
just experienced sp for the first time
Does anyone else experience this?
BF hearing voices

### I used praw to scrap up to 1000 posts from the Sleep Paralysis subreddit

In [ ]:
#spdf = []
#sp_subreddit = reddit.subreddit('Sleepparalysis')
#for post in sp_subreddit.hot(limit=1000):
#    spdf.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
#spdf = pd.DataFrame(spdf,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
#print(spdf)


 title  score      id  \
0                                       Identifying SP    543  f8d1kj   
1    NSFW: My sleep paralysis experiences with rape...     17  ieme6n   
2    The Times I Have Experienced Sleep Paralysis S...      3  iek3u9   
3    I think I may have just had a sleep paralysis ...     11  ieenbo   
4       Defying the laws of physics - sleep paralysis?     14  ie7k6n   
..                                                 ...    ...     ...   
825                           My first sleep paralysis      6  h89oa8   
826              From frightening to a more calm state      3  h84jff   
827  My first sp experience and I hope I don’t have...     34  h7nnha   
828  Anybody with recurring sleep paralyses moments...      7  h7uq2d   
829    The story on how I got stuck in my lucid dream.      2  h7x1l6   

          subreddit                                                url  \
0    Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   
1    Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   
2    Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   
3    Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   
4    Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   
..              ...                                                ...   
825  Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   
826  Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   
827  Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   
828  Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   
829  Sleepparalysis  https://www.reddit.com/r/Sleepparalysis/commen...   

     num_comments                                               body  \
0             177  I’m making this because 75% of this sub is peo...   
1               7  Hi guys, I just found this sub and I’m honestl...   
2               0  Aaaaaaaaaaaaaaaaalright I have quite a few sto...   
3               7  It’s currently 8:57 and this took place probab...   
4               4  Happened about 8 years ago but I still remembe...   
..            ...                                                ...   
825             0  It was scary at first since I have never had t...   
826             1  I remember very vividly my first encounters wi...   
827            20  I was laying face down but I could see my room...   
828             3  So I’ve had this dream about 5 times now in th...   
829             2  Usually I get sleep paralysis and it’s scary b...   

          created  
0    1.582510e+09  
1    1.598146e+09  
2    1.598138e+09  
3    1.598112e+09  
4    1.598080e+09  
..            ...  
825  1.592091e+09  
826  1.592070e+09  
827  1.592007e+09  
828  1.592030e+09  
829  1.592038e+09  

[830 rows x 8 columns]

### I wrote the raw data to a csv so I would not have to rerun everthing when I restarted my kernel.

In [ ]:
#spdf.to_csv('./data/SleepParalysisRAW.csv')

In [2]:
sleeppardf=pd.read_csv('./data/SleepParalysisRAW.csv')

### I found the nulls in the data set and filled them with 0s, and I dropped unneeded columns.

In [3]:
sleeppardf.isnull().sum()

Unnamed: 0       0
title            0
score            0
id               0
subreddit        0
url              0
num_comments     0
body            10
created          0
dtype: int64

In [4]:
sleeppardf['body'].fillna(0,inplace=True)

In [5]:
sleeppardf=sleeppardf.drop(columns=['score','url','id','created','Unnamed: 0'])

In [6]:
sleeppardf.head(5)

,title,subreddit,num_comments,body
0,Identifying SP,Sleepparalysis,177,I’m making this because 75% of this sub is peo...
1,is it normal to experience sleep paralysis mul...,Sleepparalysis,19,"for background, ive missed a couple doses of m..."
2,oh no,Sleepparalysis,0,isn’t water just a snowman’s blood?? 🤯🤯🤔🤔
3,Something that works for me with preventing SP...,Sleepparalysis,1,Not sure it will work for other people but it ...
4,The moment I fall asleep I wake up in a panic ...,Sleepparalysis,0,This has only been happening for a few months....


### I wrote the cleaned data back to a csv

In [7]:
#sleeppardf.to_csv('./data/SleepParalysisCLEANED.csv')

## NOW FOR ALIEN ABDUCTIONS

In [ ]:
#hot_posts = reddit.subreddit('Abductions').hot(limit=10)
#for post in hot_posts:
#    print(post.title)

In [ ]:
#Adf = []
#ALabduction_subreddit = reddit.subreddit('Abductions')
#for post in ALabduction_subreddit.hot(limit=1000):
#    Adf.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
#Adf = pd.DataFrame(Adf,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
#print(Adf)

                             title  score      id  \
0    25 Space Facts That Will Both Terrify And Amaz...      8  iealx8   
1                                    BF hearing voices      7  ie8owi   
2                                  Is Blue Ink a Sign?      5  icpeh4   
3    Mauritius oil spill: Experts fear catastrophic...      3  icdn52   
4             Should I continue posting Finding Truth?      7  ibiwsw   
..                                                 ...    ...     ...   
599                                     My Experiences     17  3gl7gk   
600                                         I Was Five      7  3gjk28   
601          The Zanfretta Abductions (Your thoughts?)      8  3gcbno   
602           The Hill Abduction Case (Your thoughts?)      6  3gcagq   
603                                   Aboard the ship.     11  3g95cl   

      subreddit                                                url  \
0    Abductions  https://www.igorbnews.com/2020/07/25-space-fac...   
1    Abductions  https://www.reddit.com/r/Abductions/comments/i...   
2    Abductions  https://www.reddit.com/r/Abductions/comments/i...   
3    Abductions  https://www.igorbnews.com/2020/08/mauritius-oi...   
4    Abductions  https://www.reddit.com/r/Abductions/comments/i...   
..          ...                                                ...   
599  Abductions  https://www.reddit.com/r/Abductions/comments/3...   
600  Abductions  https://www.reddit.com/r/Abductions/comments/3...   
601  Abductions  https://www.reddit.com/r/Abductions/comments/3...   
602  Abductions  https://www.reddit.com/r/Abductions/comments/3...   
603  Abductions  https://www.reddit.com/r/Abductions/comments/3...   

     num_comments                                               body  \
0               0                                                      
1               5  So my boyfriend just shared with me his experi...   
2               1  When I was a teen I would go to my dads house ...   
3               4                                                      
4              17  Let's have it already.\n\n[View Poll](https://...   
..            ...                                                ...   
599            11  It first began when I was 7 years old. One sum...   
600            10   My first memories began in this house. Centra...   
601             0  [The Zanfretta Abductions](https://www.youtube...   
602             6  [The Hill Abduction Case](https://www.youtube....   
603             6  I had an abduction experience,It started with ...   

          created  
0    1.598092e+09  
1    1.598085e+09  
2    1.597878e+09  
3    1.597827e+09  
4    1.597715e+09  
..            ...  
599  1.439327e+09  
600  1.439289e+09  
601  1.439150e+09  
602  1.439148e+09  
603  1.439078e+09  

[604 rows x 8 columns]

In [ ]:
# wrote the raw data to csv file
#Adf.to_csv('./data/AlienAbductionsRAW.csv',index=False)

In [8]:
Adf=pd.read_csv('./data/AlienAbductionsRAW.csv')

In [ ]:
Adf.head()

### I found the nulls in the data set and filled them with 0s, and I dropped unneeded columns.

In [9]:
Adf.isnull().sum()

title             0
score             0
id                0
subreddit         0
url               0
num_comments      0
body            174
created           0
dtype: int64

In [10]:
Adf['body'].fillna(0,inplace=True)

In [11]:
Adf=Adf.drop(columns=['score','id','created','url'])

In [12]:
Adf.head(5)

,title,subreddit,num_comments,body
0,25 Space Facts That Will Both Terrify And Amaz...,Abductions,0,0
1,BF hearing voices,Abductions,5,So my boyfriend just shared with me his experi...
2,Is Blue Ink a Sign?,Abductions,1,When I was a teen I would go to my dads house ...
3,Mauritius oil spill: Experts fear catastrophic...,Abductions,4,0
4,Should I continue posting Finding Truth?,Abductions,17,Let's have it already.\n\n[View Poll](https://...


In [13]:
#Adf.to_csv('./data/AlienAbductionsCLEANED.csv')

### I combined the Alien Abduction and Sleep Paralysis datasets together

In [14]:
df = pd.concat([Adf, sleeppardf])

In [15]:
df.head()

,title,subreddit,num_comments,body
0,25 Space Facts That Will Both Terrify And Amaz...,Abductions,0,0
1,BF hearing voices,Abductions,5,So my boyfriend just shared with me his experi...
2,Is Blue Ink a Sign?,Abductions,1,When I was a teen I would go to my dads house ...
3,Mauritius oil spill: Experts fear catastrophic...,Abductions,4,0
4,Should I continue posting Finding Truth?,Abductions,17,Let's have it already.\n\n[View Poll](https://...


### I got dummies for the Subreddit column and wrote the new combined df to csv

In [18]:
df=pd.get_dummies(df,columns=['subreddit'],drop_first=True)

In [20]:
df['subreddit2']=df['subreddit_Sleepparalysis']

In [21]:
df=df.drop(columns=['subreddit_Sleepparalysis'])

In [23]:
df.to_csv('./data/AlienAbductionsSleepParalysisCLEANED.csv',index=False)